# steps

1) decide on sensor locations and get lat and long

2) use geopandas to identify nearyby weights

3) Pull data

4) train model

In [3]:
# Imports
import geopandas as gpd
import numpy as np
import pandas as pd
import os 
import requests
import json
import datetime
import time
import random

# Data Pull

### Census

In [2]:
k = "5e27dc113ca0300dabe2cbb8018489d6fcaf59fa"
URL = f"https://api.census.gov/data/2017/ecnlocmfg?get=NAME,NAICS2017_LABEL,ESTAB&for=county:011&in=state:49&NAICS2017&key={k}"
response = requests.get(URL)
response.json()

### Salt Lake Infrastructure

### Tract Centers - Geopandas
I may want to add sensors nearby and weights

In [27]:
geojson_gdf = gpd.read_file('../static/data/converted_geojson_data.geojson')

geojson_gdf['centroid'] = geojson_gdf.geometry.centroid

/var/folders/kn/v0dr_8xn1qj15xgwlhdyfzph0000gn/T/ipykernel_41735/2113192064.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geojson_gdf['centroid'] = geojson_gdf.geometry.centroid


In [28]:
geojson_gdf.columns

Index(['OBJECTID', 'GEOID', 'Source', 'geoname', 'Stusab', 'Countyname',
       'State', 'County', 'Tract', 'BLKGRP', 'Low', 'Lowmod', 'Lmmi',
       'Lowmoduniv', 'Lowmod_pct', 'uclowmod', 'ucLowmod_p', 'MOE_LOWMOD_PCT',
       'MOE_UCLOWMOD_PCT', 'Shape__Area', 'Shape__Length', 'geometry',
       'centroid'],
      dtype='object')

In [29]:
geojson_gdf = geojson_gdf[['OBJECTID', 'GEOID','Countyname','Tract','Low','Lowmod','Lowmod_pct','centroid']]


In [30]:
geojson_gdf

,OBJECTID,GEOID,Countyname,Tract,Low,Lowmod,Lowmod_pct,centroid
0,199060,490111251021,Davis County,125102,120,340,0.1833,POINT (-111.88081 41.08032)
1,199061,490111251022,Davis County,125102,245,490,0.1892,POINT (-111.88004 41.04248)
2,199062,490111251031,Davis County,125103,370,850,0.3131,POINT (-111.95631 41.13919)
3,199063,490111251032,Davis County,125103,430,1000,0.4264,POINT (-111.94339 41.11548)
4,199064,490111251041,Davis County,125104,165,475,0.1967,POINT (-111.91575 41.12877)
...,...,...,...,...,...,...,...,...
907,200607,490572112012,Weber County,211201,70,170,0.1560,POINT (-111.92050 41.15201)
908,200608,490572112013,Weber County,211201,365,975,0.4194,POINT (-111.93263 41.15613)
909,200609,490572112021,Weber County,211202,115,270,0.3354,POINT (-111.96410 41.16503)
910,200610,490572112022,Weber County,211202,620,1710,0.3963,POINT (-111.95159 41.15407)


In [31]:
geojson_gdf.to_file('../static/data/centroids_data.geojson', driver='GeoJSON')

In [ ]:
geojson_gdf['latitude'] = geojson_gdf['centroid'].apply(lambda geom: geom.y)
geojson_gdf['longitude'] = geojson_gdf['centroid'].apply(lambda geom: geom.x)

### Open-Mateo
https://open-meteo.com/en/docs/

In [3]:
# Must install the following

#!pip install openmeteo-requests
#!pip install requests-cache retry-requests numpy pandas



In [121]:
dfx = pd.read_csv("../static/data/slc_daily_pm2.5_pm10_2016to2024.csv")
dfx = dfx.sort_values(['date'], ascending = False)

In [122]:
dfx=dfx.groupby(['sensor_id','latitude','longitude']).count().reset_index()[['latitude','longitude']]
dfx

,latitude,longitude
0,40.750816,-111.825290
1,41.224422,-111.968376
2,40.783870,-111.870140
3,40.595387,-111.807755
4,40.727630,-111.824425
...,...,...
217,40.593708,-111.895250
218,40.664610,-111.829270
219,40.567074,-111.988560
220,41.210533,-111.932014


In [133]:
222/6


37.0

In [135]:
for i in range (0,222,37):
    print(i)

0
37
74
111
148
185
222


In [155]:
wind_df = pd.DataFrame({})

In [156]:
#I really only needed wind for the actual sensor location for historic

# I want to modify this to just get. specific latitudes and longitudes instead of my grid method

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
start_date = datetime.datetime.strptime("2022-10-14", '%Y-%m-%d')
end_date = start_date + datetime.timedelta(days=10)

final_date = datetime.datetime.strptime("2024-03-10", '%Y-%m-%d')

i=0

while start_date < final_date:
    print(start_date)
    for i in range(0,222,37):

        url = "https://archive-api.open-meteo.com/v1/archive"

        params = {
            "latitude": list(dfx.iloc[i:i+36].latitude),
            "longitude": list(dfx.iloc[i:i+36].longitude),
            "start_date": datetime.datetime.strftime(start_date, '%Y-%m-%d'),
            "end_date": datetime.datetime.strftime(end_date, '%Y-%m-%d'),
            "daily": ["wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant",\
                     'weather_code','temperature_2m_max','temperature_2m_min',\
                     'rain_sum','snowfall_sum','precipitation_hours'],
            "temperature_unit": "fahrenheit",
            "wind_speed_unit": "mph",
            "timezone": "America/Denver"
        }
        responses = requests.get(url, params=params)
        print(responses.status_code)

        temp_df_b = pd.DataFrame()
        for response in responses.json():
    
            temp_df = pd.DataFrame(response['daily'])
            temp_df['latitiude'] = response['latitude']
            temp_df['longitude'] = response['longitude']
            temp_df['elevation'] = response['elevation']
            temp_df.drop_duplicates(subset=['wind_speed_10m_max',\
                                            'wind_gusts_10m_max',\
                                            'wind_direction_10m_dominant',\
                                            'weather_code','temperature_2m_max','temperature_2m_min',\
                                            'rain_sum','snowfall_sum','precipitation_hours',\
                                            'time'], keep='first',inplace = True)

            temp_df_b = pd.concat([temp_df_b,temp_df],ignore_index=True)

        wind_df = pd.concat([wind_df,temp_df_b],ignore_index=True)
        time.sleep(8+random.random())
    
    time.sleep(25.9+random.random())
    start_date += datetime.timedelta(days=11)
    end_date = start_date + datetime.timedelta(days=10)
    

2022-10-14 00:00:00
200
200
200
200
200
200
2022-10-25 00:00:00
200
200
200
200
200
200
2022-11-05 00:00:00
200
200
200
200
200
200
2022-11-16 00:00:00
200
200
200
200
200
200
2022-11-27 00:00:00
200
200
200
200
200
200
2022-12-08 00:00:00
200
200
200
200
200
200
2022-12-19 00:00:00
200
200
200
200
200
200
2022-12-30 00:00:00
200
200
200
200
200
200
2023-01-10 00:00:00
200
200
200
200
200
200
2023-01-21 00:00:00
200
200
200
200
200
200
2023-02-01 00:00:00
200
200
200
200
200
200
2023-02-12 00:00:00
200
200
200
200
200
200
2023-02-23 00:00:00
200
200
200
200
200
200
2023-03-06 00:00:00
200
200
200
200
200
200
2023-03-17 00:00:00
200
200
200
200
200
200
2023-03-28 00:00:00
200
200
200
200
200
200
2023-04-08 00:00:00
200
200
200
200
200
200
2023-04-19 00:00:00
200
200
200
200
200
200
2023-04-30 00:00:00
200
200
200
200
200
200
2023-05-11 00:00:00
200
200
200
200
200
200
2023-05-22 00:00:00
200
200
200
200
200
200
2023-06-02 00:00:00
200
200
200
200
200
200
2023-06-13 00:00:00
200
200
200


In [157]:
wind_df_E_2022 = wind_df
wind_df_E_2022.to_csv('wind_df_E_2023.csv')
#wind_df_C_2021 (actually 2022)
#wind_df_A_2020 = wind_df (really 2021_A)
#wind_df_B_2021
#pd.concat([wind_df_A_2020,wind_df_B_2020]).groupby('time').count().to_csv('examine.csv')

In [ ]:
#wind_df.drop_duplicates(inplace=True)
wind_df.drop_duplicates(subset=['wind_speed_10m_max',\
                                'wind_gusts_10m_max',\
                                'wind_direction_10m_dominant',\
                                'time'], keep='first',inplace = True)

In [ ]:
wind_df.rename(columns={'latitiude':'latitude'}, inplace=True)
wind_df.sort_values(['time','latitude','longitude'],inplace=True)

### Purple Air Data
if I'm using alititude

In [ ]:
dfx = pd.read_csv("../static/data/slc_daily_pm2.5_pm10_2016to2024.csv")
dfx = df.sort_values(['date'], ascending = False)


In [ ]:
df_sensors = df[['sensor_id','latitude','longitude','date','altitude','pm2.5_atm','pm10.0_atm']]
df_sensors.rename(columns = {'pm2.5_atm':'pm2','pm10.0_atm':'pm10'},inplace=True)
df_sensors['date'] = pd.to_datetime(df['date'], format='%m/%d/%y')